In [7]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from citipy import citipy

In [2]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low=-90, high=90, size=1500)
lngs = np.random.uniform(low=-180, high=180, size=1500)

lat_lngs = zip(lats, lngs)

lat_lngs

In [3]:
# Add the latitudes and longitudes to a list

coordinates = list(lat_lngs)

In [6]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

86.55385440019757 65.98727334561639
17.497306966779547 -34.79316210961238
-71.34617814582337 44.12023475000919
70.93686882487978 -89.86654640118837
84.71849465972033 153.43980203664717
17.28079737600973 47.11742562908597
-78.33873973218968 -170.37401915996705
64.63691617035042 23.952769576030903
10.39017340603418 -22.336748043316987
-55.7151479623758 -43.85000779446054
45.24520919320523 -107.68618653755644
40.87659986014498 -33.17307407203728
-61.40764065479459 60.7417556920916
53.83407023125238 -76.06470111627698
-85.74003723116454 -77.61095959020219
9.314054676510565 46.621900298024485
52.803782154437215 -91.93982116226665
35.761032100836616 -74.67762903782376
-18.775115962337196 69.12593752724229
-13.96846434211838 -69.44171130277189
70.5183791672244 126.6049820887078
87.84222275719955 141.05467789553262
14.781570545166389 73.70132642493101
15.928301792439271 19.64307090804826
24.887322527931914 -49.41872190575691
-26.5656520494028 34.33257021324229
41.631678677898805 142.3745569433

In [8]:
# Create a list to hold the cities

cities = []

# Find the nearest city for each set of coordinates

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # Add the city to the cities list if it's not already in there
    if city not in cities:
        cities.append(city)

# Print the city count
len(cities)

617